## Note : 
Mean free path : $\lambda = \frac{1}{n \sigma}$

If $n \rightarrow 100 \times n$, then :
- $\lambda \rightarrow \frac{\lambda}{100}$
- $\text{cell size} \rightarrow \frac{\text{cell size}}{100}$ ($100^2$ more cells because the grid is 2D)
- $dt \rightarrow \frac{dt}{100}$

But you can lower, down to a certain extent, the number of particles per cell to "not incease too much the computation time".

## Representative physical length scale

The system has two lenghts : 
* lenght of the tube : $L = 0.01$ $m$
* width : $w = 0.001$ $m$

The width is the representative physical length scale that constrains the flow.

So : $K_d = \frac{\lambda}{w}$.




In [1]:
%matplotlib widget

# notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as scipy
from pathlib import Path

from lppydsmc.data.saver import Saver
from plotting import analysis
from lppydsmc.utils import physics

plt.figure.max_open_warning = False

In [ ]:
def convert_to_species(row):
    if(row['species'] == 0):
        return 'I-'
    else :
        return 'I'

## Loading results

In [2]:
dir_path = Path('results/')

simu = 'dmsc_multiple_species' # low, mid, thruster_low, test_hall

dz = 0.001

In [3]:

if(simu == 'dmsc_multiple_species'): # representative lenght scale varies along the flow ...
    dt = 1e-8
    mr = 8000000 # np.array([80000000,   800000.])
    name = 'dsmc_two_species_full.h5'
    lenght = 0.018 # m
    width = 0.005
    cell_size = 0.001
    temperature = np.array([300,1000]) # K
    nb_cells = 41
    volume = cell_size*cell_size*dz*nb_cells

In [4]:
store = pd.HDFStore(dir_path/name)

In [5]:
# defining each variable
collisions_per_cell_full = store['collisions_per_cell'] 
df_full = store['df']
pmax_per_cell = store['pmax_per_cell'] # cell-discretization not taken into account 
averages_per_cell = store['averages_per_cell'] # idem
total_deleted = store['total_deleted']
total_distance = store['total_distance']
total_proba = store['total_proba']
collisions_with_walls = store['collisions_with_walls']
df_out_particles = store['df_out_particles']

In [6]:
# df_full['x'] = df_full['x'].astype(float)
# df_full['y'] = df_full['y'].astype(float)
# df_full['vx'] = df_full['vx'].astype(float)
# df_full['vy'] = df_full['vy'].astype(float)
# df_full['vz'] = df_full['vz'].astype(float)

In [7]:
choice = 'I'
gamma = 5/3. # roughly for atomes, for diatomic molecule : 7/5.

mass = physics.get_mass_part(53, 53, 74) # I
molecular_mass = 0.1269 # kg/mol
    
verbose=False

In [8]:
sound_vel = physics.speed_of_sound(molecular_mass, temperature, gamma)
print(f'Speed of sound for [{choice}] at temperature {temperature} K : {sound_vel} m/s')

Speed of sound for [I] at temperature [ 300 1000] K : [180.99192586 330.44453506] m/s


## Note : 
To save a figure, you can use : `analysis.save_fig(fig, path, title = None, dpi = 400, figsize = None)`, which allows you to easily save a figure while giving it a title and changing its size.
You can also use the interactive widget.

## Choosing the part of the system to plot 

In [9]:
# grid resolutions for the thruster

thruster = True

if(thruster):
    res_x_0, res_y_0 = 18, 5 # 3 grids
else:
    res_x_0, res_y_0 = 10, 1 # 100, 10 # 25, 10

total_nb_cells = res_x_0*res_y_0

In [10]:
fig, ax = plt.subplots(1, constrained_layout = True, figsize = (res_x_0, res_y_0));

fact = 10
volume_cell_plot = volume/(fact*fact*res_x_0*res_y_0)
duration = df_full.index.unique().values.shape[0]
analysis.hist2d(ax, df_full, bins = (fact*res_x_0,fact*res_y_0), weights = mr*np.ones(df_full.shape[0])/(duration*volume_cell_plot), stat = 'count'); # TODO: add cmap

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
use_part = False

# tube last part
if(use_part):
    if(thruster):
        part = 1 # 2
        if(part == 1):
            x_min, x_max = 0, 3e-3 
            y_min, y_max = 0.0, 5e-3
            
            # collisions
            res_x, res_y = 3, 5 
            offset = 0
            cells = [k for k in range(15)]
            
        elif(part == 2):
            x_min, x_max = 0.005, 1e-2
            y_min, y_max = 2e-3, 3e-3
            
            # collisions
            res_x, res_y = 5,1
            offset = 27
            cells = [27, 32, 37, 42, 47] 
            
        volume = (x_max-x_min)*(y_max-y_min)*dz
        df = df_full.loc[(df_full['x']>x_min) & (df_full['x']<x_max) & (df_full['y']>y_min) & (df_full['y']<y_max)].copy(deep=True)

        # in addition we want to select the right cells in collisions
        # plotting the mfp for the selected cells.
        # first along x axis, then along y axis
        nb_rows = collisions_per_cell_full.shape[0]
        
        index_collisions = collisions_per_cell_full.index.unique().values
        collisions_per_cell = np.zeros((res_x,res_y,index_collisions.shape[0]))

        for cell in cells :
            i, j = (cell-offset)//res_y_0, (cell-offset)%res_y_0
            collisions_per_cell[i,j,:] = collisions_per_cell_full[cell:nb_rows:total_nb_cells].values
    else: # tube
        hall = False
        if(not hall):
            x_min, x_max = 9e-3, 1e-2
            y_min, y_max = 0, 1e-3

            res_x, res_y = 1, 1
            offset = 9 
            cells = [9]
        else:
            x_min, x_max = 2e-2, 2.5e-2
            y_min, y_max = 0, 1e-2

            res_x, res_y = 5, 10 
            offset = 200 
            cells = [offset+k for k in range(50)] 
            
        volume = (x_max-x_min)*(y_max-y_min)*dz
        df = df_full.loc[(df_full['x']>x_min) & (df_full['x']<x_max) & (df_full['y']>y_min) & (df_full['y']<y_max)].copy(deep=True)
        # in addition we want to select the right cells in collisions
        # plotting the mfp for the selected cells.
        # first along x axis, then along y axis
            
        nb_rows = collisions_per_cell_full.shape[0]
        
        index_collisions = collisions_per_cell_full.index.unique().values
        collisions_per_cell = np.zeros((res_x,res_y,index_collisions.shape[0]))

        for cell in cells :
            i, j = (cell-offset)//res_y_0, (cell-offset)%res_y_0
            collisions_per_cell[i,j,:] = collisions_per_cell_full[cell:nb_rows:total_nb_cells].values
        
else:
    df = df_full # another reference.
    nb_rows = collisions_per_cell_full.shape[0]

    ## we are still
    index_collisions = collisions_per_cell_full.index.unique().values
    collisions_per_cell = np.zeros((res_x_0,res_y_0,index_collisions.shape[0]))
    for cell in range(total_nb_cells):
        i, j = cell//res_y_0, cell%res_y_0
        collisions_per_cell[i,j,:] = collisions_per_cell_full[cell:nb_rows:total_nb_cells].values

## Choosing frames to plot

In [11]:
unique_index = df.index.unique().values
nb_save = unique_index.shape[0]
iterations = np.max(unique_index)
adding_period = unique_index[1]-unique_index[0] # adding period - required to
# generally speaking, you choose frames so you have the steady state
frames = unique_index[int(0.8*nb_save):nb_save] 

if(verbose):
    print(f'Available frames :  {unique_index}')
    print(f'Max iteration : {iterations}')
    print(f'Choosen frames (for plotting) : \n{frames}')

## Particles analysis

In [60]:
df['species'] = df.apply(convert_to_species, axis = 1)

In [61]:
df_it = df.loc[np.isin(df.index, frames)]

In [ ]:
fig, ax = plt.subplots()
sns.despine(fig)

sns.scatterplot(
    data = df_it,
    # bins = 100,
    x="x", y ="y", hue="species", s = 5.
);

In [13]:
# scatter plot
dy = 1e-3
y_offset = 2e-3 # m

In [14]:
df_it.head()

,x,y,vx,vy,vz,species
16050,0.001955,0.000971,-62.585327,-103.458850,221.757380,0.0
16050,0.004220,0.004176,-54.921494,-32.878754,-161.362068,0.0
16050,0.001479,0.003520,-239.093290,-19.593816,-247.227958,0.0
16050,0.004604,0.002487,186.468306,40.329430,-81.653129,0.0
16050,0.001422,0.003747,8.858203,87.398795,135.424461,0.0


In [15]:
df_central = df_it.loc[(df_it['y']>y_offset) & (df_it['y']<y_offset+dy)]

In [16]:
df_central_I = df_central.loc[df_central['species']==0.0]
df_central_Im = df_central.loc[df_central['species']==1.0]

In [20]:
fig, ax = plt.subplots(constrained_layout = True)
dx, bins = 1e-3, 18 # should always be equal to 1.8e-2 m
ax.set_yscale('log')
duration_it = df_it.index.unique().values.shape[0]

analysis.hist1d(ax, df_central_I, 'x', bins = bins, weights = mr/(dy*dx*dz*duration_it)*np.ones(df_central_I.shape[0]), color = 'b', histtype = 'step', label = 'I')
analysis.hist1d(ax, df_central_Im, 'x', bins = bins, weights = mr/(dy*dx*dz*duration_it)*np.ones(df_central_Im.shape[0]), color = 'r', histtype = 'step', label = 'I-')
analysis.set_axis(ax, 'x', 'density')
ax.legend(loc='best');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Plotting density of particles evolution

In [21]:
df_species = df['species']

In [22]:
nb_parts = df_species.groupby(df_species.index).agg('count').values

In [23]:
df_I = df_species.loc[df_species==0.0]
nb_parts_I = df_I.groupby(df_I.index).agg('count').values

In [25]:
fig, ax = plt.subplots(constrained_layout = True)
ax.plot(df.index.unique()*dt, nb_parts_I*mr/volume, label = 'I')
ax.plot(df.index.unique()*dt, (nb_parts-nb_parts_I)*mr/volume, label = 'I-')
ax.set_yscale('log')
plt.legend(loc='best')
analysis.set_axis(ax, x = 'time', y = 'density')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Velocity distribution

In [32]:
df_it_I = df_it.loc[df_it['species']==0.0]
df_it_Im = df_it.loc[df_it['species']==1.0]

In [ ]:
fig, axes = analysis.velocity_distribution(df_it_Im, frames = None, bins = 100, density = True, sharex = False, sharey = False);

### stream plot

In [64]:
dp = 1e-4
df['i'] = (df['x']/dp).astype(int)
df['j'] = (df['y']/dp).astype(int)

In [80]:
groups = df.loc[df['species'] == 'I-'].groupby(['i','j']).mean()

In [81]:
groups = groups.reset_index()

In [82]:
groups.head()

,i,j,x,y,vx,vy,vz
0,0,0,0.000050,0.000050,36.467973,0.055359,2.251485
1,0,1,0.000051,0.000150,34.446578,1.366428,-1.455642
2,0,2,0.000050,0.000251,36.316271,0.479362,0.428609
3,0,3,0.000051,0.000349,31.136116,-0.375187,-3.388886
4,0,4,0.000050,0.000450,29.705002,0.907710,0.693182


# fig, ax = plt.subplots()

values = {
    'width' : 3*dp,
    'headwidth' : 3,
    'headlength' : 5
}

ax.quiver(dp*groups['i'], dp*groups['j'], groups['vx'], groups['vy'], groups['vx'], **values);

In [83]:
U,V = np.zeros((groups['j'].max()-groups['j'].min(),groups['i'].max()-groups['i'].min()), dtype = float), np.zeros((groups['j'].max()-groups['j'].min(),groups['i'].max()-groups['i'].min()), dtype = float)
X, Y = np.arange(groups['i'].min(), groups['i'].max()),  np.arange(groups['j'].min(), groups['j'].max())

for x in X:
    for y in Y:
        row = groups.loc[(groups['i'] == x) &  (groups['j'] == y)]
        try :
            U[y,x] = row['vx']
            V[y,x] = row['vy']
        except ValueError as e:
            pass

In [84]:
print(U.shape, V.shape)
print(X.shape, Y.shape)
speed = np.sqrt(U**2+V**2)
lw = 2 * speed / (speed.max()+1e-8)

(49, 179) (49, 179)
(179,) (49,)


In [85]:
fig, ax = plt.subplots(constrained_layout=True)
# ax.axis('equal')
h = ax.streamplot(x=dp*X,y=dp*Y,u=U,v=V, density = 3, color=U, linewidth=lw, cmap='plasma'); 
cbar = fig.colorbar(h.lines, ax=ax)
analysis.set_axis(ax, x = 'x', y = 'y');

/home/calot/.conda/envs/NIRN/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Out particles plot

In [37]:
df_out_particles.head()

,x,y,vx,vy,vz,species
428,0.000006,0.002304,363.439511,378.742603,-641.354589,1.0
537,0.000008,0.000532,344.426674,223.743444,-71.792443,1.0
540,0.000054,0.002455,456.420105,335.388637,-426.036197,1.0
679,0.000003,0.003318,176.077568,-91.954525,-20.691895,1.0
882,0.000008,0.001263,257.014865,-191.246538,91.309577,1.0


In [41]:
df_out_particles['species'] = df_out_particles.apply(convert_to_species, axis = 1)

In [42]:
df_out_particles['v'] = np.sqrt(df_out_particles['vx']**2+df_out_particles['vy']**2+df_out_particles['vz']**2)
df_out_particles['energy'] = df_out_particles['v']**2
df_out_particles['angle'] = np.arctan(df_out_particles['vy']/df_out_particles['vx'])

In [43]:
ions_out = df_out_particles.loc[df_out_particles['species'] == 'I']
neutral_out =  df_out_particles.loc[df_out_particles['species'] == 'I-']

In [45]:
results = df_out_particles[['species', 'x']].groupby('species').count();
try:
    print('Out particles neutralization percentage : {:.3} %'.format(100*results.iloc[1,0]/(results.iloc[1,0]+results.iloc[0,0])))
except:
    print('100% neutralization')
    print(results)

Out particles neutralization percentage : 97.9 %


In [46]:
vel_results = df_out_particles[['species', 'vx', 'vy', 'vz', 'v','energy']].groupby('species').agg('mean');
vel_results

,vx,vy,vz,v,energy
species,,,,,
I,465.607331,-15.458939,3.706414,680.171857,1.116743e+06
I-,174.748800,-0.519005,-0.514525,262.645984,7.809780e+04


In [47]:
analysis.set_fontsizes(SMALL_SIZE = 10, MEDIUM_SIZE = 12, BIGGER_SIZE = 14)

In [48]:
fig, ax = plt.subplots(constrained_layout = True)
sns.despine(fig)

sns.histplot(
    df_out_particles,
    x="vx", hue="species",
    bins = 100,
    multiple="stack",
    palette="light:m_r",
    edgecolor=".3",
    linewidth=.5,
    log_scale=False,
);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [49]:
fig, ax = plt.subplots(constrained_layout = True)
#sns.despine(fig)
ax.set_xticks([-np.pi/2, -np.pi/4, 0, np.pi/4, np.pi/2]);
ax.set_xticklabels(['-π/2', '-π/4', '0', 'π/4', 'π/2'])
sns.histplot(
    df_out_particles,
    # bins = 100,
    x="angle", hue="species",
    multiple="stack",
    palette="light:m_r",
    edgecolor=".3",
    linewidth=.5,
    log_scale=False,
)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='angle', ylabel='Count'>

### Thrust due to ions

In [52]:
print('Mean speed out : {:.3e} m/s.'.format(df_out_particles.mean()['vx']))

Mean speed out : 1.810e+02 m/s.


In [53]:
density_ion = 3.2e17
v_mean_injection_ion = 2000 # m/s
flux_ion = 0.25*density_ion*v_mean_injection_ion

density = 3.2e19
v_mean_injection = 200 # m/s
flux = 0.25*density*v_mean_injection

# in this simulation
mean_speed_ion = df_out_particles.mean()['vx']
mean_speed = 100 # m/s

surfasic_thrust_ion = mean_speed_ion*mass*flux_ion
surfasic_thrust = mean_speed*mass*flux
total_thrust = surfasic_thrust+surfasic_thrust_ion
print('Ratio ion / neutral : {:.3}'.format(surfasic_thrust_ion/surfasic_thrust))
print('Thrust per injection surface : {:.2e} N/m²'.format(total_thrust))

Ratio ion / neutral : 0.181
Thrust per injection surface : 4.01e-02 N/m²


In [54]:
surface_pegase = 5e-2 * 4e-2 # m²
print('Pegase thrust : {:.3e} N'.format(total_thrust*surface_pegase))

Pegase thrust : 8.027e-05 N


In [58]:
V_accel = 300 # V
e = 1.6e-19
print('Isp : {:.3e} s-1'.format(mean_speed_ion/9.81))
print('Theoretical Isp : {:.3e} s-1'.format((np.sqrt(2*e*V_accel/mass))/9.81))
print('Efficiency : {:.3} %'.format(100*mean_speed_ion/np.sqrt(2*e*V_accel/mass)))

Isp : 1.845e+01 s-1
Theoretical Isp : 2.167e+03 s-1
Efficiency : 0.851 %


df_mean = df.groupby(df.index).mean()
## Temperature evolution

First - not by cell but for the whole system.

### Theory : 

The kinetic energy density $w$ is

$w = \frac{3}{2}p(\vec{r},t) + n(\vec{r},t)\frac{1}{2} m |\vec{u}(\vec{r},t)|^2$ 

With $\vec{u}(\vec{r},t) = ((<(v_x, v_y, v_z)>)(\vec{r},t))^T$ thus : $|\vec{u}(\vec{r},t)| = |<\vec{v}>|$.

Here the drift is not zero. So we have to remove it to have only $\frac{3}{2}p(\vec{r},t) = w - n(\vec{r},t)\frac{1}{2} m |\vec{u}(\vec{r},t)|^2$.

This yields : 

$\frac{3}{2}p(\vec{r},t) = n(\vec{r},t)  <\frac{1}{2}mv^2> - n(\vec{r},t)\frac{1}{2} m |\vec{u}(\vec{r},t)|^2$

Which yields : 

$$ \frac{3}{2}p(\vec{r},t) = \frac{m n(\vec{r},t)}{2}  (<v^2> - |<\vec{v}>|^2) $$

With $p = nkT$.

Intuition : total energy - energy contains in the drift.

In [43]:
df['v'] = np.sqrt(df['vx']*df['vx']+df['vy']*df['vy']+df['vz']*df['vz']) # df.apply(analysis.speed_norm, axis = 1)
df['v2'] = df['v']**2
df_mean = df.groupby(df.index).mean()
df_mean['v_drift'] = np.sqrt(df_mean['vx']*df_mean['vx']+df_mean['vy']*df_mean['vy']+df_mean['vz']*df_mean['vz'])

In [44]:
# maxwellian temperature 
temperature_mb = np.pi*mass*df_mean['v']**2/(8*physics.BOLTZMAN_CONSTANT)

In [45]:
df_mean['temperature'] =  mass/(3.*physics.BOLTZMAN_CONSTANT) * (df_mean['v2']-df_mean['v_drift']**2)
temperatures = df_mean['temperature']

In [46]:
fig, ax = plt.subplots(1,2)
ax[0].plot(unique_index*dt,temperatures)
ax[1].plot(unique_index*dt,temperature_mb)

analysis.set_axis(ax[0], x = 'time', y = 'temperature')
analysis.set_axis(ax[1], x = 'time', y = 'temperature')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [47]:
fig, ax = plt.subplots()
ax.plot(unique_index*dt,temperatures)
analysis.set_axis(ax, x = 'time', y = 'temperature')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
fig, ax = plt.subplots()
ax.plot(unique_index*dt, physics.pressure_torr(nb_parts*mr[0], volume, temperature_mb)*1e3)
analysis.set_axis(ax, x = 'iteration', y = None)
ax.set_ylabel('pressure (mTorr)');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

TypeError: 'int' object is not subscriptable

In [49]:
sound_speed = physics.speed_of_sound(molecular_mass, temperature_mb, gamma)

In [42]:
fig, ax = plt.subplots()
ax.plot(unique_index*dt,sound_speed)
analysis.set_axis(ax, x = 'time', y = 'sound speed')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Flow speed

In [43]:
fig, ax = plt.subplots()
ax.plot(unique_index*dt,df_mean['v_drift'])
analysis.set_axis(ax, x = 'time', y = 'v');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Mach number

In [44]:
fig, ax = plt.subplots()
ax.plot(unique_index*dt,df_mean['v_drift']/sound_speed)
analysis.set_axis(ax, x = 'time')
ax.set_ylabel('Mach number');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Collision with walls - analysis

In [48]:
fig, ax = plt.subplots()
analysis.set_axis(ax, x = 'time', y = 'quantity')
ax.plot(collisions_with_walls.index*dt, collisions_with_walls*np.mean(mr));

/home/calot/.conda/envs/NIRN/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## DSMC collisions analysis

### Particles collisions analysis

In [51]:
# start by summing over all cells
collisions = np.sum(collisions_per_cell, axis = (0,1))
print('Total number of collision in the simulation (in reality) : {:e} ({:e})'.format(int(np.sum(collisions_per_cell)),mr[0]*np.sum(collisions_per_cell)));

Total number of collision in the simulation (in reality) : 0.000000e+00 (0.000000e+00)


In [54]:
fig, ax = plt.subplots()
analysis.set_axis(ax, x = 'time', y = 'quantity')
ax.plot(index_collisions, collisions*mr[0]);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Rate between the two

In [55]:
fig, ax = plt.subplots()
analysis.set_axis(ax, x = 'time')
ax.set_ylabel('rate')
ax.plot(index_collisions*dt, collisions_with_walls/collisions);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Mean free path 
The formula for the mean free path is : $\lambda = \frac{v_{mean}}{\nu}$, where : 
* $v_{mean}$ is the mean velocity ;
* $\nu$ is the frequency of collision for ONE particle.

We have : $\frac{1}{\nu} = \frac{N_c dt}{2 N}$ with $N_c$ the number of particle in the cell, and $N$ the number of collisions.

This yields :  $\lambda = \frac{v_{mean}N_c dt}{2N}$. There is no need for $m_r$ here as both $N_c$ and $N$ would require the multiplication to have the current number in the system.

We start by doing it for the whole system.

In [56]:
x_res = 10 # for example the number of cells during the simulations (if you want a plot per cell)
y_res = 1
x_step = 0.001 # size of the cell
y_step = 0.001

In [57]:
v_mean = df_mean['v']

In [58]:
# selecting the time iteration that we also saved for the collisions (saving frequency is not necessary the same)
v_mean = v_mean.loc[np.isin(v_mean.index,index_collisions)]
nb_particles_in_system = df['x'].groupby(df.index).agg('count').loc[np.isin(unique_index,index_collisions)]

In [59]:
mfp = physics.mean_free_path_simu(nb_particles_in_system, collisions, dt, v_mean)

### Mean free path evolution

In [60]:
fig, ax = plt.subplots()
ax.plot(index_collisions*dt, mfp)
analysis.set_axis(ax, x = 'time', y = 'mean free path')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Knudsen evolution

In [61]:
fig, ax = plt.subplots()
ax.plot(index_collisions*dt, mfp/cell_size)
analysis.set_axis(ax, x = 'time', y = 'Knudsen')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Total mean free path

Accouting for collisions between particles and **collisions with walls**.

In the case where the number of collisions with walls is much bigger than the number of collisions between particles, then this mean free path is basically the average distance a particle covers between two wall collisions.

In [ ]:
mfp_ = 0.5*nb_particles_in_system*dt*v_mean/(collisions+collisions_with_walls)

#### Total Mean free path evolution 

In [ ]:
fig, ax = plt.subplots()
ax.plot(index_collisions*dt, mfp_)
analysis.set_axis(ax, x = 'time', y = 'mean free path');

#### Total Knudsen - evolution

In [ ]:
fig, ax = plt.subplots()
ax.plot(index_collisions*dt, mfp_/cell_size)
analysis.set_axis(ax, x = 'time', y = 'Knudsen');

## Out particle analysis

In [67]:
fig, ax = plt.subplots()
mass_flow_rate = mr*physics.compute_mass_flow_rate(total_deleted, dt, mass)
analysis.set_axis(ax, x = 'time', y = 'mass flow rate')
plt.plot(mass_flow_rate.index*dt, mass_flow_rate);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ValueError: operands could not be broadcast together with shapes (2,) (21,) 

### Mean number of particles

In [65]:
averages = averages_per_cell.groupby(averages_per_cell.index).sum() # returns a pandas Series

In [66]:
fig, ax = plt.subplots()
analysis.set_axis(ax, x = 'time', y = 'quantity')
ax.plot(averages.index*dt, averages);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Mean acceptance probability

In [64]:
proba = total_proba/collisions
print('Mean proba : {:e}'.format(np.mean(proba)))
fig, ax = plt.subplots()
analysis.set_axis(ax, x = 'time', y = 'probability')
ax.plot(proba.index*dt, proba, label = 'proba');

Mean proba : nan


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Maximum probability

In [62]:
pmax = pmax_per_cell.groupby(pmax_per_cell.index).sum() # returns a pandas Series

In [63]:
fig, ax = plt.subplots()
analysis.set_axis(ax, x = 'time', y = 'probability')
ax.plot(pmax.index*dt, pmax);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Mean distance between two colliding particles

In [ ]:
fig, ax = plt.subplots()
dist = total_distance/collisions
print('Mean distance : {:e} m'.format(np.mean(dist)))
analysis.set_axis(ax, x = 'time', y = 'distance')
ax.plot(dist.index*dt, dist);

### Velocity axis distribution per *cell*

In [ ]:
# spatial_hist2d plots an hist per cell defined by the params below
# it is a 2D plot of size y_res * x_res
# where each plot (i,j) takes the particle between i*x_step < x < (i+1)*x_step and j*y_step < y < (j+1)*y_step
# for i varying between between 0 and x_res-1, and j varying between between 0 and y_res-1
# You should not use too many cells because it quickly becomes very hard to view.
# Params
x_res = 11 # for example the number of cells during the simulations (if you want a plot per cell)
y_res = 5
x_step = 0.001 # size of the cell
y_step = 0.001 

# Plotting - vx
# fig, axes = analysis.spatial_hist2d(df, frames, val = 'vx', x_res = x_res, y_res = y_res, x_step = x_step, y_step=y_step);

In [ ]:
# Plotting - vy
# fig, axes = analysis.spatial_hist2d(df, frames, val = 'vy', x_res = x_res, y_res = y_res, x_step = x_step, y_step=y_step);

## Bonus - Not necessarily recommended and takes lots of time
Next is the plot of the system for each saved step. 

In [ ]:
# creating speed_norm (for the color)
# df['speed_norm'] = np.sqrt(df['vx']**2+df['vy']**2+df['vz']**2) -> DO NOT NEED THAT

In [ ]:
%matplotlib notebook
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation

def update_hist(num, df):
    dfit = df.loc[df.index == num]

    scat.set_offsets(np.c_[dfit['x'],dfit['y']])
    scat.set_array(df['v'])
    
    ax.set_title('{}/{}'.format(num+1, iterations), fontsize=15)

fig, ax = plt.subplots()
dfit = df.loc[df.index == 0]
scat = ax.scatter(dfit['x'], dfit['y'], s=0.1, c = dfit['v'], cmap='seismic') #  c = df['speed_norm']
analysis.set_axis(ax, x = 'x', y = 'y')
# if you want the boundaries to be plotted and the grid too, you have to initialize those fields yourself.
# in the other case, only particles will be plotted.
# plot_boundaries(ax, segments)
# plot_grid(ax, resolutions, system_shape)

ax.set_title('{}/{}'.format(1, iterations), fontsize=12)
ax.axis('equal')

interval = 40 # 25 images per second
anim = FuncAnimation(fig, update_hist, interval=interval, frames=iterations, fargs=(df, ), save_count=iterations)
anim.save(dir_path/'system_state_evo.avi', dpi = 300);